In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import lseg.data as ld
from lseg.data.content import search
app_key = '<APP_KEY>'

ld.open_session(app_key=app_key)

cpn_dict = {'FXAN':'FRB', 'FXDI':'FRB', 'FXPP':'FRB', 'FXPM':'FRB', 'FXMF':'FRB', 'FXZC':'FRB', 'FXPV':'FRB', 'FRBF':'FRN', 'FRPV':'FRN', 'FRFF':'FRN', 'FXRV':'FRN', 'FROT':'FRN', 'FRFX':'FRN', 'FRVR':'FRN', 'FRFZ':'FRN', 'FRPM':'FRN', 'FRSD':'FRN', 'FRSU':'FRN', 'VRFR':'FRN', 'FRZF':'FRN', 'RGOT':'FRB', 'TBPD':'FRN', 'FTZR':'FRB', 'VRGR':'FRB', 'ZRFX':'FRB', 'ZRVR':'FRB'}

def get_bond_type(is_ilb, cpn):
    inst_type = []
    for x in range(len(is_ilb)):
        if is_ilb[x].strip() == 'Y':
            inst_type.append('ILB')
        else:
            inst_type.append(cpn_dict[cpn[x]])     
    return inst_type

def get_data(codes, at_date = ''):
    today_date = datetime.strptime(datetime.today().strftime('%Y-%m-%d'), '%Y-%m-%d')
    at_date = datetime.strptime(at_date, '%Y-%m-%d') if at_date != '' else ''
    realtime = True if (today_date == at_date or at_date == '') else False
    output = []
    crv = pd.DataFrame()    
    
    raw = ld.get_data(
        codes,
        ['PRIMACT_1', 'SEC_ACT_1', 'RT_YIELD_1', 'SEC_YLD_1', 'PRC_QL3', 'TR.FiMaturityDate', 
         'TR.ADF_BONDSTRUCTURE', 'TR.ADF_RATESTRUCTURE', 'TR.ADF_STRUCTURE', 'TR.FiCouponRate', 
         'TR.FiCurrency', 'TR.FiDescription', 'TR.FiInflationProtected', 'TR.FiCouponType', 
         'TR.ADF_MARGIN', 'TR.FiSeniorityType', 'TR.FiIsBenchmark', 'TR.UltimateParentId', 
         'TR.FiIsPutable', 'TR.FiIsCallable', 'TR.FiIsSinkable', 'TR.FiIsPerpetualSecurity', 'TR.FiNextCallDate'])
    
    
    
    
    raw['FRN Margin'] = pd.to_numeric(raw['FRN Margin'], errors='coerce')
    raw = raw.dropna(how = 'all', subset = ['PRIMACT_1', 'SEC_ACT_1', 'RT_YIELD_1', 'SEC_YLD_1'])
    raw = raw.reset_index(drop=True)
    crv['ric'] = raw['Instrument']
    tmp_ric = raw['Instrument'].values.tolist()
    
    response = search.lookup.Definition(
        view=search.Views.SEARCH_ALL,
        scope="RicRoot",
        terms=",".join(tmp_ric),
        select="ISIN",
    ).get_data()
   
    isin = response.data.df
    crv['isin'] = isin['ISIN'].values.tolist()
    crv['maturity'] = raw['Maturity Date'].dt.strftime('%Y-%m-%d')
    crv['coupon'] = raw['Coupon Rate'] / 100
    crv['currency'] = raw['Currency']
    crv['bond_structure'] = raw['Bond Structure']
    crv['rate_structure'] = raw['Rate Structure']
    crv['structure'] = raw['Structure']
    crv['description'] = raw['Description']
    crv['seniority'] = raw['Seniority Type']
    crv['putable'] = raw['Is Putable']
    crv['callable'] = raw['Is Callable']
    crv['sinkable'] = raw['Is Sinkable']
    crv['bond_type'] = get_bond_type(raw['Inflation Protected'].values.tolist(), raw['Coupon Type'].values.tolist())
    crv['quoted_margin'] = raw['FRN Margin'] / 100
    crv['quotation_method'] = raw['PRC_QL3']
    crv['issuer_permid'] = raw['Ultimate Parent Id']
    crv['is_benchmark'] = raw['Is Benchmark']
    crv['is_perpetual'] = raw['Is Perpetual Security']
    crv['call_date'] = raw['Call Date'].dt.strftime('%Y-%m-%d')
        
    if realtime:
        crv['bid_yield'] = raw['RT_YIELD_1']/100
        crv['ask_yield'] = raw['SEC_YLD_1']/100            
        crv['yield'] = crv['bid_yield']
        crv['tenor'] = (pd.to_datetime(crv['maturity'], format = '%Y-%m-%d') - datetime.now())  / np.timedelta64(1,'D')
        for i, p in enumerate(crv['is_perpetual']):
            if p == 'Y':
                try:
                    crv['tenor'][i] = (pd.to_datetime(crv['call_date'][i], format = '%Y-%m-%d') - datetime.now())  / np.timedelta64(1,'D')
                except:
                    0
        crv['tenor'] = crv['tenor'] / 365
        crv['bid'] = raw['PRIMACT_1'] / 100
        crv['ask'] = raw['SEC_ACT_1'] / 100
        
    return crv